In [1]:
import pandas as pd

In [2]:
metadata = pd.read_json('metadata.json')

In [3]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 679 entries, 0 to 678
Data columns (total 4 columns):
ac_code       679 non-null int64
ac_name       679 non-null object
state_code    679 non-null object
state_name    679 non-null object
dtypes: int64(1), object(3)
memory usage: 21.3+ KB


In [4]:
metadata.sample(3)

,ac_code,ac_name,state_code,state_name
325,18,Aizawl South i,S16,Mizoram
582,27,Choppadandi,S29,Telangana
137,49,CHANDLA,S12,Madhya Pradesh


In [5]:
df_1 = pd.read_json('assembly-elections-2018.json', lines=True)

In [6]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9150 entries, 0 to 9149
Data columns (total 5 columns):
ac       9150 non-null int64
name     9150 non-null object
party    9150 non-null object
state    9150 non-null object
votes    9150 non-null int64
dtypes: int64(2), object(3)
memory usage: 357.5+ KB


In [7]:
df_1.sample(3)

,ac,name,party,state,votes
1405,8,KAMLESH JATAV,Indian National Congress,S12,37343
8880,91,DAMODAR REDDY RAM REDDY,Indian National Congress,S29,62683
4659,40,M. CHAKHU,Mizo National Front,S16,4648


In [8]:
"df_1.shape={} metadata.shape={}".format(df_1.shape, metadata.shape)

'df_1.shape=(9150, 5) metadata.shape=(679, 4)'

In [9]:
# add state_name and ac_name columns
df_2 = df_1.merge(metadata.rename(columns={'ac_code':'ac','state_code':'state'}))

In [10]:
df_2.shape

(9150, 7)

In [11]:
df_2.sample(3)

,ac,name,party,state,votes,ac_name,state_name
8569,54,None of the Above,None of the Above,S29,1381,PARGI,Telangana
6581,5,SONA DEVI,Indian National Congress,S20,31294,Raisingh Nagar,Rajasthan
2859,116,RAJENDRA RAI,Gondvana Gantantra Party,S12,21694,KEOLARI,Madhya Pradesh


In [12]:
# number of candidates in each state
df_2.state_name.value_counts()

Madhya Pradesh    3129
Rajasthan         2473
Telangana         1940
Chhattisgarh      1359
Mizoram            249
Name: state_name, dtype: int64

In [13]:
df = df_2 # dataframe we are working on

# number of constituencies and candidates
pd.DataFrame(dict(
    constituencies=df.groupby(['state_name','ac_name'], as_index=False).first().state_name.value_counts(), 
    candidates=df.state_name.value_counts()))

,candidates,constituencies
Madhya Pradesh,3129,230
Rajasthan,2473,198
Telangana,1940,119
Chhattisgarh,1359,90
Mizoram,249,40


In [14]:
def by_state(df):
    byac = df.groupby('ac')
    byparty = df.groupby('party')
    tmp = (byparty[['votes']].sum()/df.votes.sum()).round(3).rename(columns={'votes':'vote_pct'})
    tmp = tmp.assign(
            won          =byac.apply(lambda g : g.nlargest(1,'votes')).party.value_counts(),
            candidates   =byparty.count().ac,
            runnerup     =byac.apply(lambda g : g.nlargest(2,'votes').iloc[1]).party.value_counts(),
            not_contested=(df.ac.unique().shape[0] - byparty.count().ac).map(lambda a: 0 if a < 0 else a)
        ).dropna().astype({'won':int,'runnerup':int}).sort_values(by='vote_pct', ascending=False)
    return tmp

df_2.groupby('state_name').apply(by_state)

vote_pct  candidates  \
state_name     party                                                       
Chhattisgarh   Indian National Congress                0.430          90   
               Bharatiya Janata Party                  0.330          90   
               Janta Congress Chhattisgarh (J)         0.076          57   
               Bahujan Samaj Party                     0.039          35   
Madhya Pradesh Bharatiya Janata Party                  0.410         230   
               Indian National Congress                0.409         229   
               Independent                             0.058        1094   
               Bahujan Samaj Party                     0.050         227   
               Samajwadi Party                         0.013          52   
Mizoram        Mizo National Front                     0.376          40   
               Indian National Congress                0.302          40   
               Independent                             0.229          63   
               Bharatiya Janata Party                  0.080          39   
Rajasthan      Indian National Congress                0.393         194   
               Bharatiya Janata Party                  0.388         199   
               Independent                             0.095         830   
               Bahujan Samaj Party                     0.040         189   
               Rashtriya Loktantrik Party              0.024          58   
               Communist Party of India  (Marxist)     0.012          28   
               Bhartiya Tribal Party                   0.007          11   
               Rashtriya Lok Dal                       0.003           2   
Telangana      Telangana Rashtra Samithi               0.469         119   
               Indian National Congress                0.284          99   
               Bharatiya Janata Party                  0.070         118   
               Telugu Desam                            0.035          13   
               Independent                             0.033         675   
               All India Forward Bloc                  0.008          19   

                                                    not_contested  runnerup  \
state_name     party                                                          
Chhattisgarh   Indian National Congress                         0        13   
               Bharatiya Janata Party                           0        68   
               Janta Congress Chhattisgarh (J)                 33         2   
               Bahujan Samaj Party                             55         4   
Madhya Pradesh Bharatiya Janata Party                           0       113   
               Indian National Congress                         1        98   
               Independent                                      0         5   
               Bahujan Samaj Party                              3         6   
               Samajwadi Party                                178         5   
Mizoram        Mizo National Front                              0        12   
               Indian National Congress                         0        18   
               Independent                                      0         9   
               Bharatiya Janata Party                           1         1   
Rajasthan      Indian National Congress                         5        72   
               Bharatiya Janata Party                           0       105   
               Independent                                      0        11   
               Bahujan Samaj Party                             10         2   
               Rashtriya Loktantrik Party                     141         2   
               Communist Party of India  (Marxist)            171         2   
               Bhartiya Tribal Party                          188         1   
               Rashtriya Lok Dal                              197         1   
Telangana      Telangana Rasht

In [15]:
def topk(k, df, getk, sortk):
  # add total_votes columns for each ac
  df = df.groupby('state', as_index=False).apply(
          lambda g: g.merge(g.groupby('ac').sum().rename(columns={'votes':'total_votes'}), on='ac')
       )

  def _vote_diff(g):
      vote_pct = g.votes/g.total_votes
      vote_pct_diff = vote_pct.iloc[0] - vote_pct
      vote_diff = g.votes.iloc[0] - g.votes
      return g.assign(vote_pct=vote_pct, vote_pct_diff=vote_pct_diff, vote_diff=vote_diff)
      

  df = df.groupby(['state','ac'], as_index=False, sort=False, group_keys=False).apply(
      lambda g: _vote_diff(g.nlargest(2,'votes'))).reset_index(drop=True)

  df = df.merge(getk(df, k, 'vote_pct_diff')[['state','ac']])
  
  df = df.groupby(['state_name','ac_name']).apply(
        lambda g: g.assign(sort_key=g.vote_pct_diff.iloc[0] + g.vote_pct_diff.iloc[1]))

  return sortk(df).drop(columns=['ac','state','ac_name','state_name','sort_key'])

In [16]:
# top10 wins by smallest margin
topk(10, 
    df_2, 
    lambda x,k,c: x[x.vote_pct_diff > 0.0].nsmallest(k, c),
    lambda x: x.sort_values(by=['sort_key','votes'], ascending=[True,False]))

name  \
state_name     ac_name                                          
Mizoram        Tuivawl         4           LALCHHANDAMA RALTE   
                               5              R. L. PIANMAWIA   
Rajasthan      Asind           6        JABBAR SINGH SANKHALA   
                               7                MANISH MEWARA   
Madhya Pradesh GWALIOR SOUTH   0               PRAVEEN PATHAK   
                               1        NARAYAN SINGH KUSHWAH   
Telangana      Asifabad        14                ATHRAM SAKKU   
                               15                  KOVA LAXMI   
Rajasthan      Pilibanga       10            DHARMENDRA KUMAR   
                               11                 VINOD KUMAR   
               Marwar junction 8              KHUSHVEER SINGH   
                               9            KESARAM CHOUDHARY   
Madhya Pradesh SUWASRA         2            DANG HARDEEPSINGH   
                               3   RADHESHYAM NANALAL PATIDAR   
Telangana      Ibrahimpatnam   18    MANCHIREDDY KISHAN REDDY   
                               19        MALREDDY RANGA REDDY   
               Dharmapuri      16              ESHWAR KOPPULA   
                               17         ADLURI LAXMAN KUMAR   
Chhattisgarh   Dhamtari        12      RANJANA DIPENDRA  SAHU   
                               13         GURUMUKH SINGH HORA   

                                                       party   votes  \
state_name     ac_name                                                 
Mizoram        Tuivawl         4         Mizo National Front    5207   
                               5    Indian National Congress    5204   
Rajasthan      Asind           6      Bharatiya Janata Party   70249   
                               7    Indian National Congress   70095   
Madhya Pradesh GWALIOR SOUTH   0    Indian National Congress   56369   
                               1      Bharatiya Janata Party   56248   
Telangana      Asifabad        14   Indian National Congress   65788   
                               15  Telangana Rashtra Samithi   65617   
Rajasthan      Pilibanga       10     Bharatiya Janata Party  106414   
                               11   Indian National Congress  106136   
               Marwar junction 8                 Independent   58921   
                               9      Bharatiya Janata Party   58670   
Madhya Pradesh SUWASRA         2    Indian National Congress   93169   
                               3      Bharatiya Janata Party   92819   
Telangana      Ibrahimpatnam   18  Telangana Rashtra Samithi   72581   
                               19        Bahujan Samaj Party   72205   
               Dharmapuri      16  Telangana Rashtra Samithi   70579   
                               17   Indian National Congress   70138   
Chhattisgarh   Dhamtari        12     Bharatiya Janata Party   63198   
                               13   Indian National Congress   62734   

                                   total_votes  vote_diff  vote_pct  \
state_name     ac_name                                                
Mizoram        Tuivawl         4         13211          0  0.394141   
                               5         13211          3  0.393914   
Rajasthan      Asind           6        199236          0  0.352592   
                               7        199236        154  0.351819   
Madhya Pradesh GWALIOR SOUTH   0        152430          0  0.369803   
                               1        152430        121  0.369009   
Telangana      Asifabad        14       160790          0  0.409155   
                               15       160790        171  0.408091   
Rajasthan      Pilibanga       10       226217          0  0.470407   
                               11       226217        278  0.469178   
               Marwar junction 8        164523          0  0.358132   
                               9        164523        251  0.356607   
Madhya Pradesh SUWASRA         2        206895 

In [17]:
df_2.query("ac_name=='Tuivawl'")

,ac,name,party,state,votes,ac_name,state_name
4726,7,LALCHHANDAMA RALTE,Mizo National Front,S16,5207,Tuivawl,Mizoram
4727,7,R. L. PIANMAWIA,Indian National Congress,S16,5204,Tuivawl,Mizoram
4728,7,JUDY ZOHMINGLIANI,Bharatiya Janata Party,S16,1607,Tuivawl,Mizoram
4729,7,K. ROMAWIA,Independent,S16,1083,Tuivawl,Mizoram
4730,7,H. K. HLIMPAWLTHANGA,Independent,S16,58,Tuivawl,Mizoram
4731,7,None of the Above,None of the Above,S16,52,Tuivawl,Mizoram


In [18]:
# top10 wins by largest margin
topk(10, 
    df_2, 
    lambda x,k,c: x.nlargest(k, c), 
    lambda x: x.sort_values(by=['sort_key','votes'], ascending=False))

name  \
state_name     ac_name                                            
Telangana      Siddipet        14          THANNEERU HARISH RAO   
                               15             BHAVANI MARIKANTI   
               Bahadurpura     8              MOHD. MOAZAM KHAN   
                               9          MIR INAYATH ALI BAQRI   
               Chandrayangutta 10             AKBARUDDIN OWAISI   
                               11              SHAHEJADI SAYYAD   
               Wardhanapet     18                 AROORI RAMESH   
                               19            DEVAIAH PAGIDIPATI   
               Sircilla        16   KALVAKUNTLA TARAKA RAMA RAO   
                               17  KONDAM KARUNA MAHENDER REDDY   
Rajasthan      Todabhim        6                     PRITHVIRAJ   
                               7                   RAMESH CHAND   
               Shahpura        4        KAILASH CHANDRA MEGHWAL   
                               5                MAHAVEER PRASAD   
Madhya Pradesh DABARA          0                    IMARTI DEVI   
                               1           KAPTAN SINGH SEHSARI   
Telangana      Jagtial         12                M.SANJAY KUMAR   
                               13      THATIPARTHI JEEVAN REDDY   
Madhya Pradesh KUKSHI          2   SURENDRA SINGH  HONEY BAGHEL   
                               3         VIRENDRA SINGH  BAGHEL   

                                                                    party  \
state_name     ac_name                                                      
Telangana      Siddipet        14               Telangana Rashtra Samithi   
                               15                  Telangana Jana Samithi   
               Bahadurpura     8   All India Majlis-E-Ittehadul Muslimeen   
                               9                Telangana Rashtra Samithi   
               Chandrayangutta 10  All India Majlis-E-Ittehadul Muslimeen   
                               11                  Bharatiya Janata Party   
               Wardhanapet     18               Telangana Rashtra Samithi   
                               19                  Telangana Jana Samithi   
               Sircilla        16               Telangana Rashtra Samithi   
                               17                Indian National Congress   
Rajasthan      Todabhim        6                 Indian National Congress   
                               7                   Bharatiya Janata Party   
               Shahpura        4                   Bharatiya Janata Party   
                               5                 Indian National Congress   
Madhya Pradesh DABARA          0                 Indian National Congress   
                               1                   Bharatiya Janata Party   
Telangana      Jagtial         12               Telangana Rashtra Samithi   
                               13                Indian National Congress   
Madhya Pradesh KUKSHI          2                 Indian National Congress   
                               3                   Bharatiya Janata Party   

                                    votes  total_votes  vote_diff  vote_pct  \
state_name     ac_name                                                        
Telangana      Siddipet        14  131295       167055          0  0.785939   
                               15   12596       167055     118699  0.075400   
               Bahadurpura     8    96993       130604          0  0.742650   
                               9    14475       130604      82518  0.110831   
               Chandrayangutta 10   95339       140290          0  0.679585   
                               11   15075       140290      80264  0.107456   
               Wardhanapet     18  131252       189269          0  0.693468   
                               19   32012       189269      99240  0.169135   
               Sircilla        16  125213       176630          0  0.708900   
                    

In [19]:
df_2.query("ac_name=='Siddipet'")

,ac,name,party,state,votes,ac_name,state_name
8839,33,THANNEERU HARISH RAO,Telangana Rashtra Samithi,S29,131295,Siddipet,Telangana
8840,33,BHAVANI MARIKANTI,Telangana Jana Samithi,S29,12596,Siddipet,Telangana
8841,33,NAINI NAROTHAM REDDY,Bharatiya Janata Party,S29,11266,Siddipet,Telangana
8842,33,PUSHPALATHA MEDOJU,Shramajeevi Party,S29,3206,Siddipet,Telangana
8843,33,BURRA SRINIVAAS,Telangana Inti Party,S29,1204,Siddipet,Telangana
8844,33,ALLADI SRINIVAS,Independent,S29,931,Siddipet,Telangana
8845,33,BHOGI SRINIVAS,Independent,S29,905,Siddipet,Telangana
8846,33,GYADARI JAGADISH @ JAGAN,Bahujana Left Party,S29,798,Siddipet,Telangana
8847,33,PEDDOLLA SRINIVAS,Bahujan Samaj Party,S29,718,Siddipet,Telangana
8848,33,CHINTALA MALLESHAM,Independent,S29,492,Siddipet,Telangana


In [ ]:
df = df_2
df = df.groupby(['state','ac']).apply(lambda g:
                                 g.assign(vote_pct=g.votes/g.votes.sum()))
df[df.ac_name.isin(df[df.name=='MANVENDRA SINGH'].ac_name)]

In [21]:
df.query("ac_name=='{}'".format(df[df.name.str.lower().str.contains("pilot")].ac_name.iloc[0]))

,ac,name,party,state,votes,ac_name,state_name
7114,96,SACHIN PILOT,Indian National Congress,S20,109040,Tonk,Rajasthan
7115,96,YOONUS KHAN,Bharatiya Janata Party,S20,54861,Tonk,Rajasthan
7116,96,MO ALI,Bahujan Samaj Party,S20,1785,Tonk,Rajasthan
7117,96,PANCHU,Shiv Sena,S20,1221,Tonk,Rajasthan
7118,96,AMAR SINGH CHOUDHARY,Bhartiya Jan Hitkari Party,S20,1052,Tonk,Rajasthan
7119,96,NEELIMA AMERA,Independent,S20,735,Tonk,Rajasthan
7120,96,RAMPAL,Aam Aadmi Party,S20,628,Tonk,Rajasthan
7121,96,RASHEDA MAJEED,Independent,S20,552,Tonk,Rajasthan
7122,96,MUKESH CHOUDHARY,Abhinav Rajasthan Party,S20,207,Tonk,Rajasthan
7123,96,None of the Above,None of the Above,S20,1485,Tonk,Rajasthan
